# Lesson 3: Reflection and Blogpost Writing

## Setup

In [1]:
llm_config = {"model": "gpt-3.5-turbo"}

## The task!

In [7]:
task = '''
        딥러닝.AI에 대한 간결하면서도 매력적인 블로그 게시물을 작성하세요. 블로그 게시물은 100단어 이내여야 합니다.
       '''


## Create a writer agent

In [8]:
import autogen

writer = autogen.AssistantAgent(
    name="Writer",
    system_message= "당신은 작가입니다. 주어진 주제에 대해 흥미롭고 간결한 블로그 글(제목 포함)을 작성하세요. 피드백을 바탕으로 글을 다듬고 정제된 버전을 제공해야 합니다. 추가적인 설명 없이 최종 작업물만 제출하세요.",
    llm_config=llm_config,
)

In [9]:
reply = writer.generate_reply(messages=[{"content": task, "role": "user"}])

In [10]:
print(reply)

**제목: 딥러닝.AI: 인간의 지능을 넘어**

딥러닝.AI는 현대 기술의 정점을 나타냅니다. 이 기술은 인공신경망을 사용하여 컴퓨터가 스스로 학습하고 의사결정을 내릴 수 있게 합니다. 이는 우리의 상상력을 뛰어넘어 인간의 지능을 추론하고 패턴을 인식하는 능력을 갖추었습니다. 자율 주행 자동차, 언어 번역, 음성 인식 등 다양한 분야에서 빠르게 발전하고 있습니다. 이처럼 끊임없이 진화하는 딥러닝.AI는 미래의 세계를 형성할 것입니다. 우리는 이 환상적인 기술에 주목해야 합니다.


## Adding reflection 

Create a critic agent to reflect on the work of the writer agent.

In [11]:
critic = autogen.AssistantAgent(
    name="Critic",
    is_termination_msg=lambda x: x.get("content", "").find("TERMINATE") >= 0,
    llm_config=llm_config,
    system_message="당신은 비평가입니다. 작가의 작품을 검토하고, 콘텐츠의 질을 향상시키기 위해 건설적인 피드백을 제공합니다.",
)

In [12]:
res = critic.initiate_chat(
    recipient=writer,
    message=task,
    max_turns=2,
    summary_method="last_msg"
)

Critic (to Writer):


        딥러닝.AI에 대한 간결하면서도 매력적인 블로그 게시물을 작성하세요. 블로그 게시물은 100단어 이내여야 합니다.
       

--------------------------------------------------------------------------------
Writer (to Critic):

**제목: 딥러닝.AI: 인간의 지능을 넘어**

딥러닝.AI는 현대 기술의 정점을 나타냅니다. 이 기술은 인공신경망을 사용하여 컴퓨터가 스스로 학습하고 의사결정을 내릴 수 있게 합니다. 이는 우리의 상상력을 뛰어넘어 인간의 지능을 추론하고 패턴을 인식하는 능력을 갖추었습니다. 자율 주행 자동차, 언어 번역, 음성 인식 등 다양한 분야에서 빠르게 발전하고 있습니다. 이처럼 끊임없이 진화하는 딥러닝.AI는 미래의 세계를 형성할 것입니다. 우리는 이 환상적인 기술에 주목해야 합니다.

--------------------------------------------------------------------------------
Critic (to Writer):

이 블로그 게시물은 딥러닝.AI에 대한 간략하면서도 매력적인 소개를 제공하고 있습니다. 좋은 점은 짧은 문장과 간결한 표현을 사용하여 내용을 명확하게 전달했다는 것입니다. 그러나 이 주제에 대해 논쟁적인 이슈나 추가적인 정보를 더하면 블로그를 더 흥미롭게 만들 수 있을 것입니다. 예를 들어, 딥러닝.AI가 현재 사회적인 이슈에 어떤 영향을 미치고 있는지 논의하거나 실제 사례나 응용 프로그램에 대한 예를 들어봅시다. 이렇게 함으로써 독자들이 더 깊이있는 이해를 얻을 수 있을 것입니다.

--------------------------------------------------------------------------------
Writer (to Critic):

**제목: 딥러닝.AI: 미래를 형성하는 혁신**

딥러닝.AI는 우리의 일상을 

## Nested chat

In [13]:
SEO_reviewer = autogen.AssistantAgent(
    name="SEO Reviewer",
    llm_config=llm_config,
    system_message="You are an SEO reviewer, known for "
        "your ability to optimize content for search engines, "
        "ensuring that it ranks well and attracts organic traffic. " 
        "Make sure your suggestion is concise (within 3 bullet points), "
        "concrete and to the point. "
        "Begin the review by stating your role.",
)


In [14]:
legal_reviewer = autogen.AssistantAgent(
    name="Legal Reviewer",
    llm_config=llm_config,
    system_message="You are a legal reviewer, known for "
        "your ability to ensure that content is legally compliant "
        "and free from any potential legal issues. "
        "Make sure your suggestion is concise (within 3 bullet points), "
        "concrete and to the point. "
        "Begin the review by stating your role.",
)

In [15]:
ethics_reviewer = autogen.AssistantAgent(
    name="Ethics Reviewer",
    llm_config=llm_config,
    system_message="You are an ethics reviewer, known for "
        "your ability to ensure that content is ethically sound "
        "and free from any potential ethical issues. " 
        "Make sure your suggestion is concise (within 3 bullet points), "
        "concrete and to the point. "
        "Begin the review by stating your role. ",
)

In [16]:
meta_reviewer = autogen.AssistantAgent(
    name="Meta Reviewer",
    llm_config=llm_config,
    system_message="You are a meta reviewer, you aggragate and review "
    "the work of other reviewers and give a final suggestion on the content.",
)

## Orchestrate the nested chats to solve the task

In [17]:
def reflection_message(recipient, messages, sender, config):
    return f'''Review the following content. 
            \n\n {recipient.chat_messages_for_summary(sender)[-1]['content']}'''

review_chats = [
    {
     "recipient": SEO_reviewer, 
     "message": reflection_message, 
     "summary_method": "reflection_with_llm",
     "summary_args": {"summary_prompt" : 
        "Return review into as JSON object only:"
        "{'Reviewer': '', 'Review': ''}. Here Reviewer should be your role",},
     "max_turns": 1},
    {
    "recipient": legal_reviewer, "message": reflection_message, 
     "summary_method": "reflection_with_llm",
     "summary_args": {"summary_prompt" : 
        "Return review into as JSON object only:"
        "{'Reviewer': '', 'Review': ''}.",},
     "max_turns": 1},
    {"recipient": ethics_reviewer, "message": reflection_message, 
     "summary_method": "reflection_with_llm",
     "summary_args": {"summary_prompt" : 
        "Return review into as JSON object only:"
        "{'reviewer': '', 'review': ''}",},
     "max_turns": 1},
     {"recipient": meta_reviewer, 
      "message": "Aggregrate feedback from all reviewers and give final suggestions on the writing.", 
     "max_turns": 1},
]


In [18]:
critic.register_nested_chats(
    review_chats,
    trigger=writer,
)

**Note**: You might get a slightly different response than what's shown in the video. Feel free to try different task.

In [19]:
res = critic.initiate_chat(
    recipient=writer,
    message=task,
    max_turns=2,
    summary_method="last_msg"
)

Critic (to Writer):


        딥러닝.AI에 대한 간결하면서도 매력적인 블로그 게시물을 작성하세요. 블로그 게시물은 100단어 이내여야 합니다.
       

--------------------------------------------------------------------------------
Writer (to Critic):

**제목: 딥러닝.AI: 인간의 지능을 넘어**

딥러닝.AI는 현대 기술의 정점을 나타냅니다. 이 기술은 인공신경망을 사용하여 컴퓨터가 스스로 학습하고 의사결정을 내릴 수 있게 합니다. 이는 우리의 상상력을 뛰어넘어 인간의 지능을 추론하고 패턴을 인식하는 능력을 갖추었습니다. 자율 주행 자동차, 언어 번역, 음성 인식 등 다양한 분야에서 빠르게 발전하고 있습니다. 이처럼 끊임없이 진화하는 딥러닝.AI는 미래의 세계를 형성할 것입니다. 우리는 이 환상적인 기술에 주목해야 합니다.

--------------------------------------------------------------------------------

********************************************************************************
Starting a new chat....

********************************************************************************
Critic (to SEO Reviewer):

Review the following content. 
            

 **제목: 딥러닝.AI: 인간의 지능을 넘어**

딥러닝.AI는 현대 기술의 정점을 나타냅니다. 이 기술은 인공신경망을 사용하여 컴퓨터가 스스로 학습하고 의사결정을 내릴 수 있게 합니다. 이는 우리의 상상력을 뛰어넘어 인간의 지능을 추론하고 패턴을 인식하는 능력을 갖추었습니다. 자율 주행 자동차, 언어 번역, 음성 인식 등


--------------------------------------------------------------------------------
Meta Reviewer (to Critic):

Aggregated Feedback:
1. Both SEO Specialists provided similar feedback on the importance of including relevant keywords, adding subheadings, and increasing content length for better optimization.
2. The Ethics Specialist focused on ensuring accuracy, transparency, highlighting ethical considerations, and including diverse perspectives in the content.

Final Suggestions:
1. Incorporate the relevant keywords suggested by the SEO Specialists strategically throughout the content to improve search engine visibility.
2. Utilize subheadings as recommended to make the content more reader-friendly and easier for search engines to crawl.
3. Expand on how "딥러닝.AI" is transforming different fields while ensuring accuracy and transparency about its capabilities and limitations as suggested by the Ethics Specialist.
4. Address ethical implications, ethical considerations, and include diverse

## Get the summary

In [20]:
print(res.summary)

**Title: 딥러닝.AI: 미래를 형성하는 혁명적 기술**

딥러닝.AI는 현재 인간의 능력을 뛰어넘는 기술로 떠오르고 있습니다. 이 기술은 자율 주행 자동차, 언어 번역, 음성 인식 등 다양한 분야에서 혁신을 이루어내며 미래 사회를 변화시키고 있습니다. 딥러닝.AI의 발전 과정과 현재 어떻게 다양한 분야에서 활용되고 있는지 자세히 알아보겠습니다. 또한, 이 기술이 가진 잠재적 윤리적 문제와 한계에 대해 투명하게 고찰하며, 다양한 시각을 종합하여 다가오는 딥러닝.AI의 세계를 탐험해보겠습니다.
